<a href="https://colab.research.google.com/github/Dileesha747/NLP_1/blob/master/NLP_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re
import pickle
import requests
import json


In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [4]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,classification_report
from sklearn.metrics import plot_confusion_matrix

In [28]:
df = pd.read_csv("new_dataop.txt",delimiter=';',names=['text','label'])

#dataset = pd.read_csv('Restaurant_Reviews.tsv',delimiter= '\t', quoting=3)

FileNotFoundError: ignored

In [29]:
#df = pd.concat([df_train,df_val])
df.reset_index(inplace=True,drop=True)
print("Shape of the DataFrame:",df.shape)
df.sample(5)

Shape of the DataFrame: (63, 2)


,text,label
3,my idea is to develop a online platform to stu...,1
31,we can implement a machine learning model to m...,0
7,front end of this platform is made from reactj...,0
20,streamlit django are good python based backend...,0
52,mqtt pubsub architecture can be used,0


In [8]:
def custom_encoder(df):
    df.replace(to_replace ="surprise", value =1, inplace=True)
    df.replace(to_replace ="love", value =1, inplace=True)
    df.replace(to_replace ="joy", value =1, inplace=True)
    df.replace(to_replace ="fear", value =0, inplace=True)
    df.replace(to_replace ="anger", value =0, inplace=True)
    df.replace(to_replace ="sadness", value =0, inplace=True)

In [9]:
custom_encoder(df['label'])

In [10]:
#object of WordNetLemmatizer
lm = WordNetLemmatizer()

In [11]:
def text_transformation(df_col):
    corpus = []
    for item in df_col:
        new_item = re.sub('[^a-zA-Z]',' ',str(item))
        new_item = new_item.lower()
        new_item = new_item.split()
        new_item = [lm.lemmatize(word) for word in new_item if word not in set(stopwords.words('english'))]
        corpus.append(' '.join(str(x) for x in new_item))
    return corpus

In [12]:
nltk.download("stopwords") 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
nltk.download("wordnet") 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [14]:
corpus = text_transformation(df['text'])

In [15]:
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus).toarray()
y = df.iloc[:,-1].values

In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)

In [17]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [18]:
y_pred = classifier.predict(X_test)

In [19]:
def expression_check(prediction_input):
    if prediction_input == 0:
        print("Idea is exposed")
    elif prediction_input == 1:
        print("Idea is secure")
    else:
        print("Invalid Statement.")

In [20]:
def sentiment_predictor(input):
    input = text_transformation(input)
    transformed_input = cv.transform(input).toarray()
    prediction = classifier.predict(transformed_input)
    expression_check(prediction)

In [30]:
input1 = ["my application is using html css and numpy in machine learning or"]
input2 = ["this app connects investors and innovators."]

In [31]:
sentiment_predictor(input1)
sentiment_predictor(input2)

Idea is exposed
Idea is secure


In [23]:
# Saving model to disk
pickle.dump(sentiment_predictor, open('model.pkl','wb'))

In [24]:
model = pickle.load(open('model.pkl','rb'))

In [26]:
print(model(["my application is using html css and numpy in machine learning"]))

Idea is exposed
None
